Machine Vision Lab 2

In [16]:
from PIL import Image

filename = 'D:\\UTAR\\Degree\\Y3S1\Machine Vision\\Practical\\Machine_Vision_Github\\Lab 1_Assets\\Access_Card.png'


with Image.open(filename) as img:
    img.load()

# img.format ##  format = PNG
# img.size ## size = (1280, 960)
# img.mode ## mode = RGB

display(img)